In [ ]:
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

import os
from dotenv import load_dotenv

load_dotenv()

llm_name = "gpt-4"
os.environ["OPENAI_API_KEY"]  # you should see the api key if the .env file is loaded correctly

In [99]:
# Replace with persist_directory location from 01_load_hummingbot_docs.ipnynb
persist_directory = "/Users/feng/Code/ai-assistant/vector_stores"

embedding = OpenAIEmbeddings()

vectordb = Chroma(
    collection_name="hummingbot_documentation",
    persist_directory=persist_directory,
    embedding_function=embedding
)

# Prompts

In [100]:
# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
prompt = PromptTemplate.from_template(template)

In [101]:
llm = ChatOpenAI(model_name=llm_name, temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_type="mmr", k=5, fetch_k=10),
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [102]:
query = """Can you explain me how to use the pure market making strategy?
"""

In [116]:
result = qa_chain({"query": query})

In [117]:
result["result"]

"The pure market making strategy involves creating limit orders on the maker side of a market. The strategy checks if there are existing bid and ask orders on the maker side and if any are missing, it checks if it's profitable to create one. If profitable, it calculates the optimal pricing and size and creates the orders. If an order is filled on the maker side, it immediately hedges by placing an order on the taker side to minimize market risks. Thanks for asking!"

In [105]:
len(result["source_documents"])

4

# Testing ConverstionalQARetrieval

In [106]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=vectordb.as_retriever(search_type="mmr", k=5, fetch_k=10),
    memory=memory
)

In [107]:
result_conv = qa({"question": query})

In [108]:
result_conv["answer"]

"I'm sorry, but the provided context does not contain specific information on how to use the pure market making strategy."

In [109]:
result_conv_2 = qa.invoke({"question": "How the order refresh time is going to affect my strategy?"})

In [110]:
result_conv_2["answer"]

'The order refresh time in a pure market making strategy determines how often your orders are updated. If the market price changes significantly within the refresh time, your orders may become uncompetitive, meaning they are too far from the market price to be filled. On the other hand, if the refresh time is too short, you may end up creating and cancelling orders too frequently, which could lead to higher transaction costs. Therefore, choosing an appropriate order refresh time is crucial to balance between order competitiveness and transaction costs.'

In [111]:
result_conv_2 = qa.invoke({"question": "My name is Federico Cardoso"})

In [112]:
result_conv_2["answer"]

'The text provided does not include the name of a person.'

In [113]:
result_conv_2 = qa.invoke({"question": "what's my name?"})

In [114]:
result_conv_2["answer"]

'The text provided does not mention the name of any individual in a conversation.'